In [268]:
import pandas as pd
import configparser
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import math

%load_ext autoreload
%autoreload 2
%aimport helper
%aimport import_dfs

from IPython.core.display import display, HTML
from matplotlib.pyplot import figure
from import_dfs import Datenbank
import helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [269]:
'''Datentypen
    cities: Stadtdaten
    tech_data: Technologiedaten
    improve_data: Verbesserungsfaktoren
    rf_pv_basis: Referenzstrekcen Basis
    rf_pv_pod: Referenzstrecken Pod
    rf_gv_basis: Referenzstrecken Güterverkehr Basis
    rf_gv_pod: Referenzstrecken Güterverkehr Pod
    rf_pv_routes: Strecken Personenverkehr 
    rf_gv_routes: Strecken Güterverkehr
'''

db = Datenbank()

['anpassungen_rf', 'besetzungsgrade', 'bevoelkerungs_und_emissiondaten', 'bevoelkerungsverteilung', 'gemis_data', 'informationen_rad', 'rf_gv_basis', 'rf_gv_pod', 'rf_gv_strecken', 'rf_pv_basis', 'rf_pv_pod', 'rf_pv_strecken', 'spezifischer_verbrauch', 'staedte', 'technologiedaten', 'verkehrsleistung_miv_(omm)']


In [270]:
tech_data = db.technologiedaten.drop(['verfuegbarkeit', 'auslastung'], axis=0, level=1)
fitted_index_lvl_0 = helper.lower_and_underscores(tech_data.index.levels[0].to_list())
tech_data.index.set_levels(fitted_index_lvl_0, level=0, inplace=True)  # Umbenennen der Modes

pod_modes = [n for n in fitted_index_lvl_0 if 'pod' in n]  # erstellen einer Liste mit allen Pod-Werten

idx = pd.IndexSlice
pod_df = pd.DataFrame(tech_data.loc[idx[pod_modes, :],'pod']).drop(['verfuegbarkeit', 'auslastung'], axis=0, level=1)

for x in pod_modes:
    pod_df.loc[(x, 'verbrauch'), 'pod'] = 0
    pod_df.loc[(x, 'is_elektro'), 'pod'] = False
    pod_df.loc[(x, 'transportmenge'), 'pod'] = 0  # Anzahl Personen oder Nutzlast

pod_df = pod_df.sort_index(level=0)

# Allgemeine Daten

In [271]:
allgemein = dict(
    v_mittel = 20,  # km/h
    is_elektro = False,
    kosten_diesel = 1.50, # €/l
    kosten_energie = 0.32, # €/MJ
    energiekosten_entwicklung = 1.2  # Anpassen !!!
)

# Straße
Für die Straße werden drei hauptsächliche Bewertungsmerkmale verwendet. Dies sind der spezifische Kraftstoffverbrauch je Fahrzeugkategorie und Antrieb, die Zusammensetzung der Fahrzeugkategorie und die Zunahme des Kraftstoffverbrauchs mit dem Gewicht. 

![spezifische Emissionen - ADAC Nutzfahrzeugstudie](../img/new.png)

Generell wird angenommen, dass der Kraftstoffverbrauch mit jedem zusätzlichen Kilo um 0.003 l/(km x 100 kg) (Diesel) und 0.004 l/(km x 100 kg) zunimmt Helmers, Eckard (2015): 'Die Modellentwicklung in der deutschen Autoindustrie: Gewicht contra Effizienz'). Die Zusammensetzung in den Fahrzeugkategorien wurde ebenfalls aus der Studie 'ADAC Nutzfahrzeugstudie' berechnet. Aus diesen Werten kann berechnet werden, wie viel ein Pod bei gegebenem Gewicht verbraucht. Mit den aus der Reaktionsgleichung von Benzin und Diesel gegeben Werten für CO$_2$ und sonstige Emissionen kann damit der Ausstoß an CO$_2$-Äquivalenten berechnet werden.  Dieser wird über die Verbesserungsfaktoren auf das Pod-Szenario fortgeschrieben. Zusammengefasst: 

1. Bestimmung Gewicht Pod
2. Einordnung in Gewichtsklassen (N1 und Stadtbuss)
3. Ermittlung des Kraftstoffverbrauchs (Sowohl Benzin und Diesel; für Elektrofahrzeuge kann der Wert aus der Tabelle übernommen werden)
4. Fortschreibung über Verebsserungsfaktoren (Sowohl Benzin und Diesel)
5. Berechnung des gesamt Kraftstoffverbrauchs über die Zusammensetzung der Fahrzeugkategorie
6. Umwandlung in Emissionsfaktoren

Elektrofahrzeuge haben nach einer Studie des Center Automotive Research eine Verbrauch der in einem Bereich von 300 kg nahezu identisch ist. Untersuchunge über mehr Zusatzgewicht exisitieren nicht. Daher wird angenommen, dass der Verbrauch gleich den Tabellenwerten ist. 

### Pod Small Straße
Ausgangspunkt für die Betrachtung ist ein leichtes Nutzfahrzeug. Diese haben eine durschnittliche Auslastung von 0.32 T/Fzg. Im Szenario autonom wird keine Steigerung der Auslastung vorgesehen. Ausgehend von dem Wert für eine Tonne wird nun überlegt, wie sich das Gewicht des Pods zusammensetzt. Angelehnt an den e.GO Moover soll ein solches Konzept Platz für bis zu 15 Personen (10 Sitzend, 5 Stehend) bieten. Geht man von einer durchschnittlichen Auslastung der Sitzplätze von 50 % aus, sind durchschnittlich 5 Personen im Fahrzeug.

![Vergleichs_auto](../img/ego_moover.png)

- Zusatzgewicht durch Aufbauten: 300 kg (Sitze, Stangen, Entertainment, Verkleidung, zusätzliche Klimaanlage, Türen)
- Zusatzgewicht durch Personen: 500 kg (5 PAX * 100 kg)
- Zusatzgewicht durch Schnittstellen: 100 kg
- Zusatzgewicht durch doppelte Strukturen: 300 kg

Daraus ergibt sich eine Gesamtzuladung von **1200 kg** pro Fahrzeug. Dazu kommt das Leergewicht des Fahrzeuges von ca. **2100 kg**. Die hier getroffenen Annahmen decken sich mit der Berechnung eines Pod-Gewichts von 1350 kg des SLA (Institut für Strukturmechanik und Leichtbau der RWTH).

In [272]:
# Konstanten
const = dict(
    gewicht_PAX = 80  # kg
)

# Variablen
var = dict(
    max_passagiere = 15,  # PAX
    auslastung = 1/3,  # Auslastung der Sitzplätze von 50 %
    leergewicht = 2100,  # kg
    zusatz_gewicht = 700,  #kg
    is_elektro = allgemein['is_elektro']  # Soll der Pod rein elektrisch betrieben sein?
)

name = 'pod_straße_small'

tmp = helper.calc_street_pod(name, var, const, db=db, pod_art='small')
pod_df = helper.add_tmp(name, pod_df, tmp)

Die Berechnung wird für ein Diesel Fahrzeug durchgeführt! 
Es handelt sich dabei um keinen reinen Diesel sondern inklusive Anteilen von Hybriden.
------------------------------------------
Anzahl an Passagieren: 5.0
Gesamtgewicht [kg]: 3200.0
------------------------------------------
Heute
Diesel Verbrauch [l/km]: 0.13
Elektro Verbrauch [kwh/km]: 0.33
------------------------------------------
2050
Diesel Verbrauch [l/km]: 0.07
Elektro Verbrauch [kwh/km]: 0.27
------------------------------------------
                   diesel
thg               42.8680
pm                 0.0027
nox                0.0602
energieverbrauch   0.5236
unfallrisiko       0.8732


### Pod Big Straße
Der Ausgangspunkt für den Pod Big Straße ist ein Stadtbuss angelehnt and den MAN Lion's City 12. Dies ist ein kurzer Stadtbuss, welcher konventionell mit Diesel angetrieben wird. 37 Sitzplätze - 77 Stehplätze 

Annahmen:

- Das zusätzliche Gewicht durch die Schnittstellen und doppelten Ausführungen kann im Vergleich zum Gesamtgewicht vernachlässigt werden
- Das zusätzliche Gewicht durch die Passagiere erhöht den Verbrauch und somit die Emissionen. Für diese Steigerung liegen kein Zahlenwerte vor.

Die Daten aus der ADAC-Studie werden daher für den Bus übernommen. Die Werte aus der Studie basieren auf einer Auslastung von 22 %. Wie im oben genannten Beispiel gehen wir von einer Auslastung der Sitzplätze von 50 % aus. Dies entspricht einer Auslastung von 23 % und somit nah an dem Wert von 2017.

In [273]:
# Variablen
var = dict(
    max_passagiere = 50,  # PAX - 77
    auslastung = 0.23,  # Auslastung der Sitzplätze von 50 % 
    is_elektro = allgemein['is_elektro']   # Soll der Pod rein elektrisch betrieben sein?
)

name = 'pod_straße_big'

tmp = helper.calc_street_pod(name, var, const, db=db, pod_art='big')
pod_df = helper.add_tmp(name, pod_df, tmp)

Die Berechnung wird für ein Diesel Fahrzeug durchgeführt! 
Es handelt sich dabei um keinen reinen Diesel sondern inklusive Anteilen von Hybriden.
------------------------------------------
Anzahl an Passagieren: 12.0
------------------------------------------
Heute
Diesel Verbrauch [l/km]: 0.45
Elektro Verbrauch [kwh/km]: 2.4
------------------------------------------
2050
Diesel Verbrauch [l/km]: 0.24
Elektro Verbrauch [kwh/km]: 1.97
------------------------------------------
                   diesel
thg               63.9026
pm                 0.0040
nox                0.0897
energieverbrauch   0.7805
unfallrisiko       0.3175


### Pod Cargo Small Straße
Der kleinere der beiden Cargo Pods wird eingesetzt für den direkten Lieferverkehr in Städten. Er ist aufgebaut analog zu den heutigen Transportern. Die Berechnung erfolgt analog zum Pod Small Straße entlang einer Gewichtsabschätzung und damit verbundenen Berechnung des Kraftstoffverbrauchs für die Diesel Variante. Für die Elektro Variante wird der Wert des Ökoinstitus übernommen und an das zusätzliche Gewicht über einen Vergleich mit den Dieselwerten angepasst. Angelehnt ist der Pod Cargo Small Straße an den e.GO Cargo Moover.

Gewichtsbetrachtung:

- Leergewicht: 2100 kg
- zusätzliche Aufbauten: 100 kg (Pakettregal, verstärkte Halterungen, etc.)
- Gewicht durch Schnittstellen: 100 kg
- Zusatzgewicht durch doppelte Strukturen: 300 kg

Daraus ergibt sich ein Gesamtgewicht von **m = 2600 kg**. Die max. Zuladung wird durch das max. erlaubte Gesamtgewicht von 3500 kg bestimmt. In diesem Falls somit 900 kg. Die gewichtstechnische Auslastung bei Transportern ist in der Regel sehr gering (300 kg) aufgrund der meist sperrigen Stückguttransporte. Die Auslastung wird übernommen bzw. sukzessive erhöht. 

In [274]:
#Konstanten
const = dict(
    max_gewicht = 3500  # max. Gewicht für Typklasse N1
)

# Variablen
var = dict(
    auslastung = 1/2,  # Auslastung von 50 %
    leergewicht = 2100,  # kg
    zusatz_gewicht = 500,  # kg
    is_elektro = allgemein['is_elektro']   # Soll der Pod rein elektrisch betrieben sein?
)

name = 'pod_straße_cargo_small'

tmp = helper.calc_street_pod(name, var, const, db=db, pod_art='cargo small')
pod_df = helper.add_tmp(name, pod_df, tmp)

Die Berechnung wird für ein Diesel Fahrzeug durchgeführt! 
Es handelt sich dabei um keinen reinen Diesel sondern inklusive Anteilen von Hybriden.
------------------------------------------
Max. Zuladung [kg]: 900
Zuladung [kg]: 450.0
Gesamtgewicht [kg]: 3050.0
------------------------------------------
Heute
Diesel Verbrauch [l/km]: 0.12
Elektro Verbrauch [kwh/km]: 0.31
------------------------------------------
2050
Diesel Verbrauch [l/km]: 0.06
Elektro Verbrauch [kwh/km]: 0.25
------------------------------------------
                    diesel
thg               408.2667
pm                  0.0253
nox                 0.5733
energieverbrauch    4.9867
unfallrisiko        0.8732


### Pod Cargo Big Straße
Der große Transporter ist angelehnt an einen mittleren Lkw der Klasse n3 (alle bisherigen E-LKWs). Für diesen sind aus der oben gennanten Shell-Studie leider keine Werte für einen elektrischen Antrieb identifiziert. 

Als Referenzauto wird daher der EMOSS CM1611 gewählt. Dieser wird von der italenischen Firma Niinivirta hergestellt und seit dem letzten Jahr ausgelierfert.

![Vergleichs_auto](../img/EMOSS_electric_truck_Finland_Niinivirta.jpg)

Gewichtsbetrachtung:
- Leergewicht: 16000 kg
- Zusatzgewicht durch zusätzliche Struktur: 300 kg
- Zusatzgewicht durch Schnittstellen: 100 kg
- Maximale Nutzlast: 7000 kg

Leistungsbetrachtung:
- Batteriekapazität: 160 kWh
- durchschnittlicher Verbrauch: 130 kWh (Aufgrund geringen Anteil Stadtverkehr)

In [275]:
#Konstanten
const = dict(
    max_gewicht = 23000  # max. Gewicht für EMOSS CM1611
)

# Variablen
var = dict(
    auslastung = 3/4,  # Auslastung von 50 %
    leergewicht = 16000,  # kg
    zusatz_gewicht = 400,  # kg
    is_elektro = allgemein['is_elektro']   # Soll der Pod rein elektrisch betrieben sein?
)

name = 'pod_straße_cargo_big'

tmp = helper.calc_street_pod(name, var, const, db=db, pod_art='cargo big')
pod_df = helper.add_tmp(name, pod_df, tmp)

Die Berechnung wird für ein Diesel Fahrzeug durchgeführt! 
Es handelt sich dabei um keinen reinen Diesel sondern inklusive Anteilen von Hybriden.
------------------------------------------
Max. Zuladung [kg]: 6600
Zuladung [kg]: 4950.0
Gesamtgewicht [kg]: 21350.0
------------------------------------------
Heute
Diesel Verbrauch [l/km]: 0.34
Elektro Verbrauch [kwh/km]: 1.3
------------------------------------------
2050
Diesel Verbrauch [l/km]: 0.18
Elektro Verbrauch [kwh/km]: 1.07
------------------------------------------
                    diesel
thg               111.3455
pm                  0.0069
nox                 0.1564
energieverbrauch    1.3600
unfallrisiko        0.1191


## Kosten der Straßen-Carrier
Die Kosten werden unabhängig von den Emissionen berechnet. Die Ermittlung der Kosten für die Pods erfolgt in sechs Schritten:
1. Bestimmung Anschaffungskosten für Pod und Carrier
2. Lineare Abschreibung des Pods über 300.000 km(≜ 6 Jahren a 50.000 km)
3. Berechnung des Mehrpreises der Carrier durch die Schnittstellen und Automatisierung im Vergleich zum Referenzszenarios
4. Bestimmung der neuen Carrier-Kosten pro Pod-km (Anteil Anschaffungskosten an Vollkosten * Mehrpreisfaktor * Kosten pro Pod-km (Szenario OMM))
5. Subtraktion durch Personenzahl
6. Kosten für die Carrier (bezogen auf Pkm) + Kosten für die Pods (bezogen auf Tkm)

Die Betriebskosten im Busverkehr ergeben sich aus: 
$ K_{Betreiber} = K_{Fahrzeug} + K_{Antrieb} + K_{Instandhaltung} + K_{Reifen} + K_{Overhead}$ 

Wobei sich die verschiedenen Bestandteile wie folgt zusammensetzen:

In der Tabelle sind die Parameter für die Berechnung abgebildet:

**kalk. Kosten**

$ K_{Fahrzeug} = \left( \frac{(K_{Anschaffung} - R) \cdot (1 + p)^{a} \cdot p}{(1 + p)^{a} - 1} + R \cdot p \right) / a$

mit: 
- $ K_{Anschaffung} $: Mittlere Anschaffungskosten eines Fahrzeugs \[€\]
- $R$: Mittlerer Restwert eines Fahrzeugs nach der Nutzungs-dauer \[€\]
- $p$: Kalkulatorischer Zinsatz \[\%\]
- $a$: Nutzungsdauer

**Treibstoffkosten/ Energiekosten**

$K_{Antrieb} = e_{energieverbrauch} \cdot k_{Antriebskosten}$

mit:
- $e_{energieverbrauch} $: Mittlerer Treibstoffverbrauch \[l/100km oder kwh/100km \]
- $k_{Antriebskosten} $ Kosten je Einheit \[ €/l oder €/kwh \]

**Instandhaltung**

$ K_{Instandhaltung} = w \cdot k_{Personal, Instandhaltung} \cdot f_g \cdot N $ 

mit:
- $w$: Personal-Kennzahl (nach VDV 881) \[ Mannjahr/Fzg/a \]
- $N$: Jahreslaufleistung je Fahrzeug \[ km/a \]
- $k_{Personal, Instandhaltung} $: jährliche Personalkosten für die Instandhaltung \[ €/(Mannjahr a) \]
- $f_g$: Faktor zur Berücksich-tigung der Fahrplangeschwindigkeit \[ - \]

**Reifenkosten**

$K_{Reifen} = \frac{k_{reifen} \cdot l}{a}  $

mit: 
- $k_{reifen}$: Jährliche Kosten für das Reifenmanagement \[ €/a \]

**Overhead**

$K_{Overhead} = \frac{k_{Overhead, Basis} \cdot f_v \cdot l}{a} $

mit:
- $k_{Overhead, Basis}$: Basiskostensatz für den Overhead \[ €/km \]
- $f_v$: Faktor zur Berücksichtigung des Verkehrsgebietes \[ - \]

Da die Preise ausgehend von heutigen Fahrzeugen berechent werden, müssen Automatisierungskosten und die Inflation mit berücksichtigt werden. Annahmen:
- Automatisierungskosten entsprechen 1.2 * aktuellen Fahrzeugwert (vgl. Bösch et al. (2018) Cost-based analysis of autonomous mobility services, S. 79)
- Aufgrund der Tatsache, dass die meiste Technik des Pods im Carrier liegt, wird angenommen, dass die Kosten des Carriers 90 % der gesamten Anschaffungskosten eines aktuellen Fahrzeugs entsprechen. 
- Es wird angenommen, dass bis auf die Personalkosten, alle Kostenpunkte einer heutigen Kostenrechnung identisch bleiben (Wartung, Trassenkosten, Reinigung, etc.) -> Daher werden diese Werte von einem Bus für die Straßen gebundenen Carrier und für die Schiene von einem Nahverkehrszug übernommen. Eine Aufsplitung in Nah- und Fernverkehr wurde in der Literatur nicht gefunden.
- Es wird von einer festgelegten Verteilung der Pods auf einem Schienencarrier ausgegangen. 

## Kosten für die Schinen Carrier
Bei der Schiene gestaltet sich die Berechnung komplexer, da weniger Daten öffentlich zugänglich sind. Es wird daher von den aktuellen Kosten ausgegangen und diese als gleichbleibend betrachtet. Einsparungen werden über den Wegfall der Personalkosten realisiert zusätzlich geht die Auslastung als variable Größe in das Modell ein. Ein Zug wird dabei immer identisch aufgeteilt. Die Aufteilung kann als variable angepasst werden. Die Kosten werden übernommen:

- Trassentgelt: 4,86 €/Zugkm
- Stationsentgelt: 1,20 €/Zugkm
- Energiekosten: 1,97 €/Zugkm -> muss angepasst werden
- Reinigung und Instandhaltung: 1,51 €/Zugkm
- Verwaltung, Wagnis und Gewinn: 0,91 €/Zugkm
- Kapitalkosten: 3,33 €/Zugkm -> muss angepasst werden


In [276]:
#Konstanten
const = dict(
    automatisierungsfaktor = 1.2,
    anteil_fahrzeugkosten_an_vollkosten_bus = 0.14,  # Übernommen aus HAASE Fahrzeugkostenkalkulation
    anteil_fahrzeugkosten_an_vollkosten_schiene = 0.22,  # Übernommen von https://www.lnvg.de/spnv/finanzierung/kosten-im-spnv
)

var = dict(
    # Pod Werte
    costs_pod_small = 15000,  # Kosten geschätzt durch SLA (evtl. zu niedrig) um Klimaanlage 1000 € erweitert
    costs_pod_big = 44000,  # Kosten auf Basis der SLA Daten und Kosten für den Carrier big fortgeschrieben  um Klimaanlage 1000 € erweitert
    costs_pod_cargo_small = 9400,  # Kosten auf Basis der SLA Kosten
    costs_pod_cargo_big = 32000,  # Schätzung basierend auf den restlichen Werten
    
    # Werte für die Straßencarrier
    costs_carrier_street_small = 71400, # Kosten für einen e.GO Moover (60 %)
    ratio_carrier_normal_vehicle_small = 0.6,
    costs_carrier_street_big = 200000,  # Kosten für eine durchschnittlichen LKW gem. HAASE Fahrzeugkostenkalkulation (60 %) Leuthardt
    ratio_carrier_normal_vehicle_big = 0.6,
    
    # Werte für die Schienencarrier
    trassentgelt = 4.86,  # €/Zugkm
    stationsentgelt = 1.20,  # €/Zugkm
    energiekosten = 1.97,  # €/Zugkm
    reinigung_und_instandhaltung = 1.51,  # €/Zugkm
    verwaltung_und_gewinn = 0.91,  # €/Zugkm
    kapialkosten = 3.33,  # €/Zugkm
    ratio_kapital_kosten = 0.5,  # ??? Hier ist eine genaure Abschätzung notwendig
    
    # Kosten je Schnittstelle
    schnittstelle = 1500,  # Schätzung des SLA 15000
    anzahl_schnittstellen_pod = 8,
    anzahl_schnittstellen_carrier = 4,
    anzahl_schnittstellen_carrier_schiene = 80,
    
    # Verteilung der Slots auf Zug: L_zug = 105 m L_pod,s = 5m Lpod,b = 10m -> 20 Slots a 5 m inklusive Abstand von 0.75 cm zwischen Pods und 7 Meter für Windbreaker
    anzahl_pod_plaetze = 20,
    anzahl_pod_s = 7 ,
    anzahl_pod_b = 3 ,
    anzahl_pod_s_cargo = 3 ,
    anzahl_pod_b_cargo = 2 ,
    
    # Allgemeine Daten Straße
    nutzungsdauer = 12,
    restwert = 10,  # % des Anschaffungswertes
    kalk_zinsatz = 5,  # klak. Zinsatz in %
    jahreskilometer = 60000,  # km pro Pod und Carrier por Jahr
    kosten_instandhaltung_elektro = 15700, # Nach Schätzungen des Frauenhofer-Institus und der Hochschule Niederein [€/a]
    kosten_instandhaltung_diesel = 13360,  # Nach Schätzungen des Frauenhofer-Institus und der Hochschule Niederein  [€/a]
    versicherungskosten = 2000, # Versicherungskosten pro Jahr nach Leuthardt
    anzahl_fahrzeuge = 140,  # Pro Verwaltungseinheit

)
        
tmp = helper.calc_costs(const, var, allgemein, pod_df, tech_data)


pod_straße_small
Anschaffungskosten [€]: 	 	  27000.0
Anteil Schnittstellen [%]:  	 	  44.0
-----------------------------------------------------------
Kalkulatorische Kosten [€/km]: 	 	  0.08
Energie Kosten [€/km]: 	 	 	  0.0
Reifen Kosten [€/km]: 	 	 	  0.0
Instandhaltung Kosten [€/km]: 	 	  0.22
Overhead Kosten [€/km]: 	 	  0.0
Versicherung Kosten [€/km]: 	 	  0.03
-----------------------------------------------------------
Gesamtkosten [€/km]: 	 	 	  0.33
spez. Gesamtkosten [€/Pkm, €/Tkm]: 	  0.07

pod_straße_big
Anschaffungskosten [€]: 	 	  56000.0
Anteil Schnittstellen [%]:  	 	  21.0
-----------------------------------------------------------
Kalkulatorische Kosten [€/km]: 	 	  0.16
Energie Kosten [€/km]: 	 	 	  0.0
Reifen Kosten [€/km]: 	 	 	  0.0
Instandhaltung Kosten [€/km]: 	 	  0.22
Overhead Kosten [€/km]: 	 	  0.0
Versicherung Kosten [€/km]: 	 	  0.03
-----------------------------------------------------------
Gesamtkosten [€/km]: 	 	 	  0.41
spez. Gesamtkosten [€/Pkm, €/

## Schiene
Für die Schiene gestaltet sich die Berechnung schwieriger, da Werte für Referenzfahrzeuge nicht vorhanden sind. Es muss daher versucht werden, über einen normalen Nahverkehrszug anhand der Auslastung, des Leergewichts und der Anzahl an Pods auf so einem Zug zu ermitteln, wie hoch der Energiebedarf ist. Der Energiebedarf kann dann mit der Auslastung und den spez. Emissionsfaktoren für 2050 umgerechnet werden. 

1. Es werden die Werte aus dem Szenario autonom übernommen
2. Dies werden von Pkm/km auf Zug/km über die durschnittlichen Passagierzahlen übertragen
3. Über Pod-Plätze auf die Pods umrechnen

**TODO**:
Mittwoch:
- Schinenfahrzeuge sich was überlegen
WE:
- Auswertungsdiagramme automatisieren 2

Donnerstag:
- typischer Strecke POD 
- Grafiken anpassen
- Streckentabellen Einheiten: m -> km; min -> std
- Tabellen für jedes Verkehrsmittel in PPT 2